In [ ]:
%install '.package(path: "$cwd/FastaiNotebooks")' FastaiNotebooks

Installing packages:
	.package(path: "/home/ubuntu/notebooks/swift/FastaiNotebooks")
		FastaiNotebooks
With SwiftPM flags: []
Working in: /tmp/tmpcan16m_x
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 2.12s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebooks' (3 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


In [ ]:
import FastaiNotebooks

In [ ]:
// export
import Path
import TensorFlow

## The forward and backward passes

In [ ]:
// export
public func normalize(_ x:Tensor<Float>, mean:Tensor<Float>, std:Tensor<Float>) -> Tensor<Float> {
    return (x-mean)/std
}

In [ ]:
var (xTrain, yTrain, xValid, yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst")

There is no std method to Tensor in swift yet, so we add it as an extension:

In [ ]:
//export
public extension Tensor where Scalar:FloatingPoint {
    func std() -> Tensor<Scalar> {
        return sqrt(self.variance(alongAxes: Array(self.shape.indices)).flattened()[0])
    }
}

Normalize the training and validation sets.

In [ ]:
let trainMean = xTrain.mean()
let trainStd  = xTrain.std()

In [ ]:
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [ ]:
//export
public func testNearZero(_ a:Tensor<Float>, tolerance:Float=1e-3) {
    assert(Raw.abs(a)<tolerance, "Near zero: \(a)")
}

In [ ]:
testNearZero(xTrain.mean())
testNearZero(xTrain.std() - 1.0)

: 

In [ ]:
xTrain.shape

In [ ]:
let (n,m) = (xTrain.shape[0],xTrain.shape[1])
let c = yTrain.max()+1
print(n,m,c)

## Foundations version

### Basic architecture

In [ ]:
//num hidden
let nh:Int32 = 50

In [ ]:
// simplified kaiming init / he init
let w1:Tensor<Float> = Tensor(randomNormal: [m,nh]) / sqrt(Float(m))
let b1:Tensor<Float> = Tensor(repeating: 0.0, shape: [nh])
let w2:Tensor<Float> = Tensor(randomNormal: [nh,1]) / sqrt(Float(nh))
let b2:Tensor<Float> = Tensor(repeating: 0.0, shape: [1])

In [ ]:
testNearZero(w1.mean())
testNearZero(w1.std()-1/sqrt(Float(m)))

In [ ]:
// This should be ~ (0,1) (mean,std)...
(xValid.mean(),xValid.std())

▿ 2 elements
  - .0 : 0.006017743
  - .1 : 1.0076997


In [ ]:
func lin(_ x:Tensor<Float>, _ w:Tensor<Float>, _ b:Tensor<Float>) ->Tensor<Float> {return matmul(x, w) + b}

In [ ]:
let t = lin(xValid, w1, b1)

In [ ]:
//...so should this, because we used kaiming init, which is designed to do this
(t.mean(),t.std())

▿ 2 elements
  - .0 : -0.12648587
  - .1 : 0.98553497


In [ ]:
func myRelu(_ x:Tensor<Float>) -> Tensor<Float> {return max(x, 0)}

In [ ]:
let t = myRelu(lin(xValid, w1, b1))

In [ ]:
//...actually it really should be this!
(t.mean(),t.std())

▿ 2 elements
  - .0 : 0.33128858
  - .1 : 0.5400599


In [ ]:
// kaiming init / he init for relu
let w1:Tensor<Float> = Tensor(randomNormal: [m,nh]) * sqrt(2.0/Float(m))

In [ ]:
(w1.mean(),w1.std())

▿ 2 elements
  - .0 : -5.9843685e-05
  - .1 : 0.050710604


In [ ]:
let t = myRelu(lin(xValid, w1, b1))
(t.mean(),t.std())

▿ 2 elements
  - .0 : 0.48134696
  - .1 : 0.7690659


In [ ]:
func model(_ xb: Tensor<Float>) -> Tensor<Float>{
    let l1 = lin(xb, w1, b1)
    let l2 = myRelu(l1)
    let l3 = lin(l2, w2, b2)
    return l3
}

In [ ]:
time(repeating: 10) {let _ = model(xValid)}

1.0305193000000001 ms


### Loss function

In [ ]:
let preds = model(xTrain)

In [ ]:
// export
func mse(_ out:Tensor<Float>, _ targ:Tensor<Float>) -> Tensor<Float> {
    return (out.squeezingShape(at: -1) - targ).squared().mean()
}

In [ ]:
var yTrainF = Tensor<Float>(yTrain)
var yValidF = Tensor<Float>(yValid)

In [ ]:
mse(preds, yTrainF)

31.965393


### Gradients and backward pass

In [ ]:
class TensorWithGrad {
    var inner: Tensor<Float>
    var grad:  Tensor<Float>
    
    init(_ x: Tensor<Float>) {
        inner = x
        grad = Tensor(repeating: 0.0, shape:x.shape)
    } 
}

In [ ]:
func lin(_ x:TensorWithGrad, _ w:TensorWithGrad, _ b:TensorWithGrad) -> TensorWithGrad {
    return TensorWithGrad(matmul(x.inner, w.inner) + b.inner)
}
func myRelu(_ x:TensorWithGrad) -> TensorWithGrad {return TensorWithGrad(max(x.inner, 0))}
func mse(_ inp: TensorWithGrad, _ targ : Tensor<Float>) -> Tensor<Float>{
    //grad of loss with respect to output of previous layer
    return (inp.inner.squeezingShape(at: -1) - targ).squared().mean()
}

In [ ]:
func mseGrad(_ inp: TensorWithGrad, _ targ : Tensor<Float>){
    //grad of loss with respect to output of previous layer
    inp.grad = 2.0 * (inp.inner.squeezingShape(at: -1) - targ).expandingShape(at: -1) / Float(inp.inner.shape[0])
}

In [ ]:
func reluGrad(_ inp:TensorWithGrad, _ out:TensorWithGrad){
    //grad of relu with respect to input activations
    inp.grad = (inp.inner .> 0).selecting(out.grad, Tensor<Float>(repeating:0.0, shape:inp.inner.shape))
}

In [ ]:
func linGrad(_ inp:TensorWithGrad, _ out:TensorWithGrad, _ w:TensorWithGrad, _ b:TensorWithGrad){
    //grad of relu with respect to input activations
    inp.grad = matmul(out.grad, w.inner.transposed())
    w.grad = matmul(inp.inner.transposed(), out.grad)
    b.grad = out.grad.sum(squeezingAxes: 0)
}

In [ ]:
let w1a = TensorWithGrad(w1)
let b1a = TensorWithGrad(b1)
let w2a = TensorWithGrad(w2)
let b2a = TensorWithGrad(b2)

In [ ]:
func forwardAndBackward(_ inp:TensorWithGrad, _ targ:Tensor<Float>){
    //forward pass:
    let l1 = lin(inp, w1a, b1a)
    let l2 = myRelu(l1)
    let out = lin(l2, w2a, b2a)
    //we don't actually need the loss in backward!
    let loss = mse(out, targ)
    
    //backward pass:
    mseGrad(out, targ)
    linGrad(l2, out, w2a, b2a)
    reluGrad(l1, l2)
    linGrad(inp, l1, w1a, b1a)
}

In [ ]:
let inp = TensorWithGrad(xTrain)

In [ ]:
forwardAndBackward(inp, yTrainF)

Let's compare to swift autodiff now:

In [ ]:
@differentiable
func foward(_ inp:Tensor<Float>, _ targ:Tensor<Float>, w1:Tensor<Float>, b1:Tensor<Float>, 
            w2:Tensor<Float>, b2:Tensor<Float>) -> Tensor<Float>{
    let l1 = matmul(inp, w1) + b1
    let l2 = relu(l1)
    let l3 = matmul(l2, w2) + b2
    return (l3.squeezingShape(at: -1) - targ).squared().mean()
}

In [ ]:
let xGrad = gradient(at: xTrain) {xTrain in foward(xTrain, yTrainF, w1:w1, b1:b1, w2:w2, b2:b2)}
let w1Grad = gradient(at: w1) {w1 in foward(xTrain, yTrainF, w1:w1, b1:b1, w2:w2, b2:b2)}
let b1Grad = gradient(at: b1) {b1 in foward(xTrain, yTrainF, w1:w1, b1:b1, w2:w2, b2:b2)}
let w2Grad = gradient(at: w2) {w2 in foward(xTrain, yTrainF, w1:w1, b1:b1, w2:w2, b2:b2)}
let b2Grad = gradient(at: b2) {b2 in foward(xTrain, yTrainF, w1:w1, b1:b1, w2:w2, b2:b2)}

In [ ]:
test_near_zero(xGrad - inp.grad)
test_near_zero(w1Grad - w1a.grad)
test_near_zero(b1Grad - b1a.grad)
test_near_zero(w2Grad - w2a.grad)
test_near_zero(b2Grad - b2a.grad)

In [ ]:
time(repeating: 10) { forwardAndBackward(inp, yTrainF) }

23.5187637 ms


### Export

In [ ]:
notebookToScript(fname: (Path.cwd / "02_fully_connected.ipynb").string)